Some of these boxes are heavy duty (for the pc) and some of these are just light enough

In [2]:
# machine translation; USE THE PC FOR THIS
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-tl")

print(pipe("This is a job for me."))

Device set to use cpu


[{'translation_text': "Ito'y isang trabaho para sa akin."}]


In [14]:
import tika
from tika import parser
import os

print(os.listdir("./booksEpub"))

epubList = []

for i in os.listdir("./booksEpub"):
    filePath = "./booksEpub/" + i
    epubList.append(parser.from_file(filePath))

['Ang_Bangka_ni_Tatay_Mario___Filipino-v12023.04.12T012033.epub', 'Ang_Paglalaro_nina_Mina_at_Manay___Filipino-v12023.06.16T022744.epub', 'Ano_ang_Kinatatakutan_nina__Anopol_at_ni_Tang-id___Filipino-v12023.06.20T194135.epub', 'Luna_Papelastika___Filipino-v12023.04.07T035616.epub', 'Misteryo_ng_Itlog___Filipino.epub', 'Pwedeng-pwede,_Pani!___Filipino-v12023.04.07T033002.epub', 'Sino_ang_Magiging_Titser_Ko_sa_Pasukan____Filipino-v12023.11.14T034013.epub', 'Si_Kokoy_at_ang_Umang-umang___Filipino-v12023.04.07T084156.epub', 'Si_Pogi,_Si_Pogs_At_Ang_Tora-tora___Filipino-v12023.11.14T082955.epub', 'Super_Wanda___Filipino-v12023.04.07T025901.epub']


In [9]:
import pymupdf
import os

print(os.listdir("./booksEpub"))

ebookContent = {}

for i in os.listdir("./booksEpub"):
    ebookText = ""
    titlePageText = ""
    filePath = "./booksEpub/" + i
    doc = pymupdf.open(filePath)
    for chapter in range(doc.chapter_count):
        print(f"Chapter {chapter}")
        if chapter==0:
            # get the texts and it will be the key
            currentPage = doc[(0,0)]
            titlePageText = currentPage.get_text()
        else:
            for page in range(doc.chapter_page_count(chapter)):
                print(f"page {page}")
                currentPage = doc[(chapter, page)]
                currentText = currentPage.get_text()
                print(currentText)
                # check if it's the last few pages
                if not(("Brought to you by" in currentText) or ("International License" in currentText) or ("Table of Contents" in currentText) or ("This book was developed as part of the ABC" in currentText) or ("Advancing Basic Education in the Philippines" in currentText)):
                    ebookText+=currentText
                else:
                    pass
    
    #print(ebookText)
    ebookContent[titlePageText] = ebookText




['Ang_Bangka_ni_Tatay_Mario___Filipino-v12023.04.12T012033.epub', 'Ang_Ngiti_ni_Ningning___Filipino-v12023.04.14T122136.epub', 'Ang_Paglalaro_nina_Mina_at_Manay___Filipino-v12023.06.16T022744.epub', 'Ang_Palakaibigang_Pusa___Filipino-v12023.04.07T084920.epub', 'Ano_ang_Kinatatakutan_nina__Anopol_at_ni_Tang-id___Filipino-v12023.06.20T194135.epub', 'Kablag!!_Krassshh!!___Filipino.epub', 'Luna_Papelastika___Filipino-v12023.04.07T035616.epub', 'Mag-aaral_na_si_Bentong___Filipino-v12023.04.07T034734.epub', 'Misteryo_ng_Itlog___Filipino.epub', 'Paano_Nagkatubig_sa_Sumagang____Filipino-v12023.07.27T073312.epub', 'Pwedeng-pwede,_Pani!___Filipino-v12023.04.07T033002.epub', 'Sa_Bayan_ng_Kamayanan_at_ang_Alamat_ng_mga_Sumasayaw_na_Kamay___Filipino-v12023.11.14T082535.epub', 'Sino_ang_Magiging_Titser_Ko_sa_Pasukan____Filipino-v12023.11.14T034013.epub', 'Si_Kokoy_at_ang_Umang-umang___Filipino-v12023.04.07T084156.epub', 'Si_Lilly,_Si_Langit_at__ang_Tatlong_Higante___Filipino.epub', 'Si_Lisa_at_Sara_

Chapter 1
MuPDF error: syntax error: css syntax error: expected number (OEBPS/style.css:6) (...icode-range: U+1000-109F, U>+<A9E0-A9FF, U+AA60-AA7F; /* ...)

page 0
Malamig na hangin ang humahampas sa balat ni Tatay
Mario.Inaayos niya ang kaniyang bangka galing sa pangingisda sa
dagat ng Roxas City. Ang kanilang bahay ay malapit sa tabing-
dagat. Kalmado ang dagat, pahiwatig na maganda ang araw.

Chapter 2
MuPDF error: syntax error: css syntax error: expected number (OEBPS/style.css:6) (...icode-range: U+1000-109F, U>+<A9E0-A9FF, U+AA60-AA7F; /* ...)

page 0
Noong araw na iyon, marami siyang nahuli na isda. May managat,
sapsap, bulawbulaw, at tabagak na sagana sagana ang Roxas. Bilang
isang mangingisda, binubuhay niya ang kaniyang limang taong
gulang na anak na si Ali.

Chapter 3
MuPDF error: syntax error: css syntax error: expected number (OEBPS/style.css:6) (...icode-range: U+1000-109F, U>+<A9E0-A9FF, U+AA60-AA7F; /* ...)

page 0
Si Ali ay masayang nakikipaglaro sa kaniyang mga kaibi

In [10]:
import re

for key in ebookContent.keys():
    print(key)
    processedKey = re.sub(r'[\.]','',key)
    processedKey = re.sub(r' ','_',processedKey)
    processedKey = re.sub(r'\?','',processedKey)
    processedKey = re.sub(r'\n','_',processedKey)
    print(processedKey)
    f = open(f"./textData1_pymupdf/{processedKey}.txt","w",encoding='utf-8')
    #print(ebookContent[key])
    f.writelines(ebookContent[key])
    f.close()
    #print("DONE")

Ang Bangka ni Tatay Mario
Jay C. Blancaflor

Ang_Bangka_ni_Tatay_Mario_Jay_C_Blancaflor_
Ang Ngiti ni Ningning
Jessa Payofelin
Justin Loise Madriaga

Ang_Ngiti_ni_Ningning_Jessa_Payofelin_Justin_Loise_Madriaga_
Ang Paglalaro nina Mina at
Manay
Anuradha
Anuradha
Bandana Tulachan

Ang_Paglalaro_nina_Mina_at_Manay_Anuradha_Anuradha_Bandana_Tulachan_
Ang Palakaibigang Pusa
Bonifacio Glenn Rivera
Joel Rosel Alcantara

Ang_Palakaibigang_Pusa_Bonifacio_Glenn_Rivera_Joel_Rosel_Alcantara_
Ano ang Kinatatakutan nina
Anopol at ni Tang-id
Kent De Lima
Valdemar Avisado

Ano_ang_Kinatatakutan_nina_Anopol_at_ni_Tang-id_Kent_De_Lima_Valdemar_Avisado_
Kablag!! Krassshh!!

Kablag!!_Krassshh!!_
Luna Papelastika
Clinton D. Abilong
Maia Hermosa

Luna_Papelastika_Clinton_D_Abilong_Maia_Hermosa_
Mag-aaral na si Bentong

Mag-aaral_na_si_Bentong_
Misteryo ng Itlog

Misteryo_ng_Itlog_
Paano Nagkatubig sa
Sumagang?
Pejay Padrigon
Allan Victor Misalucha

Paano_Nagkatubig_sa_Sumagang_Pejay_Padrigon_Allan_Victor_Mi

In [8]:
import os
import re

sourceFolder = "./textData1_pymupdf"
textNum = 1

# format the text correctly first
for i in os.listdir(sourceFolder):
    filePathForTokenizingText = f"{sourceFolder}/{i}"
    f = open(filePathForTokenizingText,'r', encoding='utf-8')
    insideText = f.read()
    f.close()

    #formatting with regex

    insideText = re.sub(r'[.]','. ',insideText)
    insideText = re.sub('\n',' ',insideText)
    insideText = re.sub('- ','-',insideText)
    insideText = re.sub('  ',' ',insideText)
    insideText = re.sub('&nbsp;','',insideText)
    

    f2 = open("./textData_forTokenization/text"+str(textNum), "w", encoding='utf-8')
    f2.writelines(insideText)
    f2.close()

    textNum+=1
    

making the vocabulary and the tokens

In [11]:
import os

postagged_filepath = "./textData_postagged/"

vocabulary = set()

for i in os.listdir(postagged_filepath):
    f = open(f"{postagged_filepath}{i}", "r", encoding="utf-16")
    story = f.read()
    
    for token in story.split():
        vocabulary.add(token)

In [14]:
import pandas as pd

pd.set_option('display.max_rows', 100)
vocab_df = pd.DataFrame(list(vocabulary), columns=['tokens'])
vocab_df['lemma'] = vocab_df['tokens'].apply(lambda x: x.split('|')[0])
vocab_df['postag'] = vocab_df['tokens'].apply(lambda x: x.split('|')[1])
vocab_df

,tokens,lemma,postag
0,Ididikit|NNP,Ididikit,NNP
1,pating|NNC,pating,NNC
2,pansit|JJD,pansit,JJD
3,Anopol|NNP,Anopol,NNP
4,pangisda|NNC,pangisda,NNC
...,...,...,...
1423,dahil|CCR,dahil,CCR
1424,libot|FW,libot,FW
1425,batang|JJD_CCP,batang,JJD_CCP
1426,pinatawa|VBTS,pinatawa,VBTS


In [15]:
pd.set_option('display.max_rows', None)
print(vocab_df.loc[ vocab_df['postag']=="NNC" ]['tokens'].shape)
vocab_df.loc[ vocab_df['postag']=="NNC" ]['tokens']

(304,)


1             pating|NNC
4           pangisda|NNC
9               baka|NNC
10               uod|NNC
14            hangin|NNC
18             tatay|NNC
21             plano|NNC
22               ΓÇ¥|NNC
30             tanim|NNC
34            bitbit|NNC
36            tutubi|NNC
51            titser|NNC
53              isda|NNC
54             galaw|NNC
56        magkakampi|NNC
57             papel|NNC
59           iskuter|NNC
64        paghakbang|NNC
66               nun|NNC
69        papelastik|NNC
73              agos|NNC
82             porma|NNC
89              taas|NNC
95             lilim|NNC
103         kalayuan|NNC
107           itsura|NNC
111            parte|NNC
115         kaarawan|NNC
117            silip|NNC
123          pamilya|NNC
125         paaralan|NNC
129            Tatay|NNC
132            balat|NNC
135             kape|NNC
141             Ewan|NNC
144         alimasag|NNC
147            kagat|NNC
154            barko|NNC
155           sarili|NNC
163           kwento|NNC


now to put these things into embeddings. Either the ELMO from tensorflowhub or the spacy torchBILSTMencoder

In [3]:
# spacy torch bilstm endcoder
import spacy
import torch




ModuleNotFoundError: No module named 'calamancy'

Notes for future me (Jan 31 2025)
- Spacy has a thing called an entity_ruler where it is a rule based entity recognition.
- depending on the pos tags, rules based category sorter can absolutely help
- then after the rule based models, we can go pure machine learning process
- then we just scale up afterward
- I'm hungry so I'll use this as a mental checkpoint
